In [1]:
from video import hazard_detection as hd
from importlib import reload
from matplotlib.pyplot import plot
import matplotlib.pyplot as plt
import os
import numpy as np 
from video.reader import VideoReader
from video import videolike as vdl
import cv2 as cv

In [2]:
directory_path = "videos"

# get the list of filenames in the directory
filenames = [directory_path+'/' + f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

Test if the red color takes > 25% of screen

In [7]:
reload(hd)

<module 'video.hazard_detection' from '/Users/nadina/Documents/codeup/epilator/video/hazard_detection.py'>

In [8]:
for f in filenames:
    print(f)
    sec = hd.detect_red_light(f)
    print(sec)

videos/Color Flash.mp4
[0]
videos/Paparazzi flashes.mp4
[]
videos/Delete & Killshot - Disco Weapon [Epilepsy Warning].mp4
[0]
videos/Blue Green Flash.mp4
[]
videos/little hens and dog.mp4
[]
videos/Red Spin.mp4
[0]
videos/Banned Pokemon Seizure Scene.mp4
[0]
videos/Superman 80th Anniversary Animated Short  @dckids.mp4
[0]
videos/Car flips over bridge following police pursuit.mp4
[]
videos/Top 10 Dangerous Lightning Strikes Thunder recorded on Camera (HIGH VOLTAGE!!).mp4
[]
videos/MASSIVE gas factory explosion caught on camera!!.mp4
[0]
videos/Plant explosion caught on camera.mp4
[]
videos/Black White Flash.mp4
[]
videos/Blue Green Fade.mp4
[]
videos/W I L T Short Art Film by Anthony Petano  EPILEPSY WARNING.mp4
[0]
videos/Clutrrglych - •Ô•‘ [OFFICIAL MUSIC VIDEO] [PHOTOSENSITIVE EPILEPSY WARNING].mp4
[0]
videos/video to induce seizures.mp4
[0]
videos/Color Fade.mp4
[0]
videos/[Epilepsy Warning]  Royal & The Serpent - Overwhelmed  Fan Music Video.mp4
[0]
videos/Learn A To Z Insects For 

13+ min to analyze videos and works incorrectly. Trying again an old func to count proportion of red color frames

In [18]:
def get_red_mask(img):
    # mask for hue below 10 hue, lightness, saturation
    lb = np.array([0,50,50], dtype=np.uint8)
    ub = np.array([10,255,255], dtype=np.uint8)
    mask1 = cv.inRange(img, lb, ub)

    # # mask for hue above 340
    lb1 = np.array([170,50, 50], dtype=np.uint8)
    ub1 = np.array([180,255, 255], dtype=np.uint8)
    mask2 = cv.inRange(img, lb1, ub1)

    return mask1 | mask2

def detect_red_frames_portion(video_path: str, seconds: bool=True):
    '''
    detects what % of video frames have red color on more than 25% of the screen
    '''
    # get video and fps
    vid, fps = VideoReader.get_vid(video_path, 
                conversion=cv.COLOR_BGR2HLS)
    # get # of frames, width and height of each frame
    width = vid.shape[2]
    height = vid.shape[1]
    frames = vid.shape[0]

    # create en empty array of integers
    narr = np.empty((0,), dtype=np.uint8)
    for v in vid:
        # get video frame
        frame = v.reshape(height, width, 3)
        # apply red mask
        mask = get_red_mask(frame)
        # append masked values as 1D array narr
        narr = np.concatenate([narr, mask.reshape(-1)])
    # narr -> array whith masked red values. represent pixels of each video frame
    # in the video. if the pixel is red it equals 1 else 0
    narr = np.where(narr == 255, 1, 0)
    # count the number of pixels in frame
    total_pixels_per_frame = width * height
    
    # reshape the narr array to the shape of (frame, total pixels)
    narr = narr.reshape(frames, total_pixels_per_frame)

    red_frames = 0
    sec = np.empty(0, dtype=int)
    for i, n in enumerate(narr):
        if n.sum() > total_pixels_per_frame / 4:
            red_frames += 1
            sec = np.append(sec, i // fps)
    
    if seconds:
        return np.unique(sec)
    else:
        return round(red_frames / frames * 100, 2)

In [19]:
for f in filenames:
    print(f)
    print(detect_red_frames_portion(f))
   

videos/Color Flash.mp4
[0 1 2 3 4]
videos/Paparazzi flashes.mp4
[]
videos/Delete & Killshot - Disco Weapon [Epilepsy Warning].mp4
[ 60  74  75  81  82 113 114 119 121 122 123 124 125 126 127 130 153 155
 156 157 158 159 160 164 165 166 167 168 174 175 176 177 178 179 180 188
 190 191 194 195 196 203 204 205 206 208 210 211 212 213 214 215 216 217
 218 220 221 222 223 225 231 232]
videos/Blue Green Flash.mp4
[]
videos/little hens and dog.mp4
[]
videos/Red Spin.mp4
[0 1 2 3 4 5 6 7 8 9]
videos/Banned Pokemon Seizure Scene.mp4
[ 8  9 15 16 17 18 19 20 22]
videos/Superman 80th Anniversary Animated Short  @dckids.mp4
[  3   4  15  21  22  23  24  25  28  29  40  54  58  59  60  77  80  81
  82  83  86  87  88  92  93  94  95  96  97  98 101 102 103 104 105 106
 107 108]
videos/Car flips over bridge following police pursuit.mp4
[]
videos/Top 10 Dangerous Lightning Strikes Thunder recorded on Camera (HIGH VOLTAGE!!).mp4
[]
videos/MASSIVE gas factory explosion caught on camera!!.mp4
[ 0  1  2 